# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_EXCERTO,TEXTO_EXCERTO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995438,Voto:Cuidam os autos de tomada de contas espec...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025603,Voto:Cuidam os autos de Solicitação do Congres...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455375,Relatório:Trata-se de embargos de declaração o...,Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773747,Voto:8. Em relação a outros processos judiciai...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773403,Voto:11. Relativamente ao ato envolvendo a Sra...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13285, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          553
Contrato Administrativo     941
Convênio                    683
Desestatização              139
Direito Processual         1811
Finanças Públicas           328
Gestão Administrativa       338
Licitação                  2756
Pessoal                    3393
Responsabilidade           2343
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 2000
dim_vetor = 50

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

Using TensorFlow backend.


Found 63925 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 2000)


In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos-50.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 63925
Encontrados no modelo: 43317 = 67.76222135314822


# Treinamento

## Embedding com pesos fixos

In [11]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W1117 00:17:51.941356 140000866477888 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W1117 00:17:52.125793 140000866477888 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1117 00:17:52.127617 140000866477888 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1117 00:17:52.134553 140000866477888 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1117 00:17:52.135160 140000866477888 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
flatten_1 (Flatten)          (None, 100000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102401024 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total para

In [13]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1117 00:17:53.013916 140000866477888 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 42s 4ms/step - loss: 10.6104 - categorical_accuracy: 0.3196 - val_loss: 12.3811 - val_categorical_accuracy: 0.2258
Epoch 2/20
10628/10628 [==============================] - 39s 4ms/step - loss: 8.7665 - categorical_accuracy: 0.4399 - val_loss: 11.4844 - val_categorical_accuracy: 0.2826
Epoch 3/20
10628/10628 [==============================] - 38s 4ms/step - loss: 7.9682 - categorical_accuracy: 0.4812 - val_loss: 11.0354 - val_categorical_accuracy: 0.2921
Epoch 4/20
10628/10628 [==============================] - 38s 4ms/step - loss: 7.2036 - categorical_accuracy: 0.5191 - val_loss: 7.5173 - val_categorical_accuracy: 0.4904
Epoch 5/20
10628/10628 [==============================] - 38s 4ms/step - loss: 6.4057 - categorical_accuracy: 0.5533 - val_loss: 6.7656 - val_categorical_accuracy: 0.5107
Epoch 6/20
10628/10628 [==============================] - 38s 4ms/step - loss: 4.5257 - cate

In [14]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 40s 4ms/step - loss: 5.2506 - categorical_accuracy: 0.4085 - val_loss: 1.6466 - val_categorical_accuracy: 0.4046
Epoch 2/20
10628/10628 [==============================] - 37s 3ms/step - loss: 1.5094 - categorical_accuracy: 0.5586 - val_loss: 1.4162 - val_categorical_accuracy: 0.5055
Epoch 3/20
10628/10628 [==============================] - 37s 3ms/step - loss: 1.2129 - categorical_accuracy: 0.6371 - val_loss: 1.5414 - val_categorical_accuracy: 0.5247
Epoch 4/20
10628/10628 [==============================] - 37s 3ms/step - loss: 1.0359 - categorical_accuracy: 0.6899 - val_loss: 1.5297 - val_categorical_accuracy: 0.5092
Epoch 5/20
10628/10628 [==============================] - 37s 3ms/step - loss: 0.8719 - categorical_accuracy: 0.7328 - val_loss: 1.3021 - val_categorical_accuracy: 0.5661
Epoch 6/20
10628/10628 [==============================] - 37s 3ms/step - loss: 0.7316 - categori

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 21s 2ms/step - loss: 3.9157 - categorical_accuracy: 0.4060 - val_loss: 1.6383 - val_categorical_accuracy: 0.4881
Epoch 2/20
10628/10628 [==============================] - 20s 2ms/step - loss: 1.5258 - categorical_accuracy: 0.5427 - val_loss: 1.8379 - val_categorical_accuracy: 0.4366
Epoch 3/20
10628/10628 [==============================] - 20s 2ms/step - loss: 1.3096 - categorical_accuracy: 0.6056 - val_loss: 1.5748 - val_categorical_accuracy: 0.5344
Epoch 4/20
10628/10628 [==============================] - 20s 2ms/step - loss: 1.1571 - categorical_accuracy: 0.6393 - val_loss: 1.7844 - val_categorical_accuracy: 0.4595
Epoch 5/20
10628/10628 [==============================] - 20s 2ms/step - loss: 1.0337 - categorical_accuracy: 0.6664 - val_loss: 1.5174 - val_categorical_accuracy: 0.5009
Epoch 6/20
10628/10628 [==============================] - 20s 2ms/step - loss: 0.9619 - categori

## Embedding com pesos ajustáveis

In [11]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1117 10:14:36.655619 140013061760832 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1117 10:14:36.723059 140013061760832 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1117 10:14:36.724688 140013061760832 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1117 10:14:36.731681 140013061760832 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
flatten_1 (Flatten)          (None, 100000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102401024 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total para

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 43s 4ms/step - loss: 6.9221 - categorical_accuracy: 0.3678 - val_loss: 3.2957 - val_categorical_accuracy: 0.3271
Epoch 2/20
10628/10628 [==============================] - 41s 4ms/step - loss: 1.7262 - categorical_accuracy: 0.5305 - val_loss: 1.6838 - val_categorical_accuracy: 0.4174
Epoch 3/20
10628/10628 [==============================] - 41s 4ms/step - loss: 1.3036 - categorical_accuracy: 0.6047 - val_loss: 1.7041 - val_categorical_accuracy: 0.4163
Epoch 4/20
10628/10628 [==============================] - 41s 4ms/step - loss: 1.1045 - categorical_accuracy: 0.6588 - val_loss: 1.5067 - val_categorical_accuracy: 0.5081
Epoch 5/20
10628/10628 [==============================] - 41s 4ms/step - loss: 0.9246 - categorical_accuracy: 0.7083 - val_loss: 1.4935 - val_categorical_accuracy: 0.5363
Epoch 6/20
10628/10628 [==============================] - 41s 4ms/step - loss: 0.8328 - categori

In [13]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 24s 2ms/step - loss: 3.4576 - categorical_accuracy: 0.4319 - val_loss: 3.3102 - val_categorical_accuracy: 0.4430
Epoch 2/20
10628/10628 [==============================] - 23s 2ms/step - loss: 1.7714 - categorical_accuracy: 0.5438 - val_loss: 2.4365 - val_categorical_accuracy: 0.4588
Epoch 3/20
10628/10628 [==============================] - 23s 2ms/step - loss: 1.5024 - categorical_accuracy: 0.5978 - val_loss: 2.5750 - val_categorical_accuracy: 0.4005
Epoch 4/20
10628/10628 [==============================] - 23s 2ms/step - loss: 1.3180 - categorical_accuracy: 0.6360 - val_loss: 2.4205 - val_categorical_accuracy: 0.4189
Epoch 5/20
10628/10628 [==============================] - 23s 2ms/step - loss: 1.1830 - categorical_accuracy: 0.6635 - val_loss: 1.9996 - val_categorical_accuracy: 0.5017
Epoch 6/20
10628/10628 [==============================] - 23s 2ms/step - loss: 1.0874 - categori